In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import logging
import math  # Add this import statement

# Set up logging
logging.basicConfig(level=logging.INFO)

# Function to extract chords from JSON file
def extract_chords_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        songs_data = data.get('songs', [])
        chord_sequences = {}
        for song in songs_data:
            title = song.get('title', 'Unknown')
            measures = song.get('music', {}).get('measures', [])
            chords = [chord for measure in measures for chord in measure]
            chord_sequences[title] = chords
        return chord_sequences

# Path to your JSON file in Google Drive
json_file_path = '/content/drive/My Drive/Colab Notebooks/MARG/irealPro/playlist.json'

# Extract chord sequences
chord_sequences = extract_chords_from_json(json_file_path)

print(chord_sequences)

{'9.20 Special': ['D9', 'F-6', 'D9', 'F-6', 'C', 'C7', 'B7', 'Bb7', 'A7', 'D9', 'G7', 'Ab7', 'G7', 'D9', 'F-6', 'D9', 'F-6', 'C', 'C7', 'B7', 'Bb7', 'A7', 'D9', 'G7', 'C6', 'C7', 'C7', 'F6', 'F6', 'D7', 'D7', 'G7', 'G7', 'D9', 'F-6', 'D9', 'F-6', 'C', 'C7', 'B7', 'Bb7', 'A7', 'D9', 'G7', 'C6'], '26-2': ['F^7', 'Ab7', 'Db^7', 'E7', 'A^7', 'C7', 'C-7', 'F7', 'Bb^7', 'Db7', 'Gb^7', 'A7', 'D-7', 'G7', 'G-7', 'C7', 'F^7', 'Ab7', 'Db^7', 'E7', 'A^7', 'C7', 'C-7', 'F7', 'Bb^7', 'Ab7', 'Db^7', 'E7', 'A^7', 'C7', 'F^7', 'C-7', 'F7', 'E-7', 'A7', 'D^7', 'F7', 'Bb^7', 'Eb-7', 'Ab7', 'Db^7', 'G-7', 'C7', 'F^7', 'Ab7', 'Db^7', 'E7', 'A^7', 'C7', 'C-7', 'F7', 'Bb^7', 'Ab7', 'Db^7', 'E7', 'A^7', 'C7', 'F^7'], '52nd Street Theme': ['C', 'A-7', 'D-7', 'G7', 'C', 'A-7', 'D-7', 'G7', 'C', 'A-7', 'D-7', 'G7', 'C', 'G7', 'C', 'C', 'A-7', 'D-7', 'G7', 'C', 'A-7', 'D-7', 'G7', 'C', 'A-7', 'D-7', 'G7', 'C', 'G7', 'C', 'C7', 'C7', 'F6', 'F6', 'D7', 'D7', 'G7', 'G7', 'C', 'A-7', 'D-7', 'G7', 'C', 'A-7', 'D-7', 

In [ ]:
# Prepare data for training
all_chords = [chord for chords in chord_sequences.values() for chord in chords]
label_encoder = LabelEncoder()
encoded_chords = label_encoder.fit_transform(all_chords)

sequence_length = 4
X, y = [], []

for chords in chord_sequences.values():
    song_encoded_chords = label_encoder.transform(chords)
    for i in range(len(song_encoded_chords) - sequence_length):
        X.append(song_encoded_chords[i : i + sequence_length])
        y.append(song_encoded_chords[i + sequence_length])

X = torch.LongTensor(X)
y = torch.LongTensor(y)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the Transformer-based Chord Generator
class TransformerChordGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, nhead, num_encoder_layers, dim_feedforward):
        super(TransformerChordGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=num_encoder_layers)
        self.fc_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(embedding_dim)  # Scale embedding by sqrt(d_model)
        src = src.permute(1, 0, 2)  # Reshape for transformer (S, N, E)
        output = self.transformer_encoder(src)
        output = self.fc_out(output)
        return output[-1]  # Return only the output of the last token

# Model, loss function, and optimizer setup with Transformer parameters
vocab_size = len(label_encoder.classes_)
embedding_dim = 50
nhead = 5
num_encoder_layers = 3
dim_feedforward = 256

model = TransformerChordGenerator(vocab_size, embedding_dim, nhead, num_encoder_layers, dim_feedforward)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop remains mostly unchanged
for epoch in range(100):
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

# Generate Sequence Function needs slight modification for Transformer
def generate_sequence(seed_text, n_words, model, label_encoder):
    model.eval()
    in_text = label_encoder.transform(seed_text).tolist()
    result = []
    for _ in range(n_words):
        in_tensor = torch.LongTensor([in_text]).to(device)
        with torch.no_grad():
            output = model(in_tensor)
        next_chord_index = torch.argmax(output, dim=1).item()
        next_chord = label_encoder.inverse_transform([next_chord_index])[0]
        result.append(next_chord)
        in_text.append(next_chord_index)
        in_text.pop(0)
    return ' '.join(result)

# Example usage remains the same
# Example usage
seed_chords = ['D-7', 'G7', 'C^7', 'A7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(generated_sequence)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 0, Loss: 2.8412153720855713
Epoch 1, Loss: 2.707437038421631
Epoch 2, Loss: 2.085359573364258
Epoch 3, Loss: 3.8548214435577393
Epoch 4, Loss: 2.7722954750061035
Epoch 5, Loss: 2.6482419967651367
Epoch 6, Loss: 3.0882198810577393
Epoch 7, Loss: 2.942753791809082
Epoch 8, Loss: 3.0643768310546875
Epoch 9, Loss: 2.265601634979248
Epoch 10, Loss: 2.371178150177002
Epoch 11, Loss: 2.8872172832489014
Epoch 12, Loss: 2.4668264389038086
Epoch 13, Loss: 2.1783225536346436
Epoch 14, Loss: 3.455939769744873
Epoch 15, Loss: 1.359692931175232
Epoch 16, Loss: 1.5970155000686646
Epoch 17, Loss: 1.5389117002487183
Epoch 18, Loss: 3.620065450668335
Epoch 19, Loss: 2.316545009613037
Epoch 20, Loss: 2.5566418170928955
Epoch 21, Loss: 1.8357369899749756
Epoch 22, Loss: 2.5384433269500732
Epoch 23, Loss: 2.312871217727661
Epoch 24, Loss: 1.6273447275161743
Epoch 25, Loss: 1.4764673709869385
Epoch 26, Loss: 1.61447012424469
Epoch 27, Loss: 1.7526346445083618
Epoch 28, Loss: 1.4913235902786255
Epoch 2

KeyboardInterrupt: 

In [ ]:
# Generate Sequence Function needs slight modification for Transformer
def generate_sequence(seed_text, n_words, model, label_encoder):
    model.eval()
    in_text = label_encoder.transform(seed_text).tolist()
    result = []
    for _ in range(n_words):
        in_tensor = torch.LongTensor([in_text]).to(device)
        with torch.no_grad():
            output = model(in_tensor)
        next_chord_index = torch.argmax(output, dim=1).item()
        next_chord = label_encoder.inverse_transform([next_chord_index])[0]
        result.append(next_chord)
        in_text.append(next_chord_index)
        in_text.pop(0)
    return ' '.join(result)



# Example usage
seed_chords = ['D-7', 'G7', 'C^7', 'A7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['D-7', 'G7', 'G-7', 'C7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['F^7', 'Eh7', 'A7', 'D-7']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

seed_chords = ['D-6', 'D-6', 'D-6', 'D-6']
generated_sequence = generate_sequence(seed_chords, 20, model, label_encoder)
print(seed_chords, generated_sequence)

['D-7', 'G7', 'C^7', 'A7'] D-7 G7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7
['D-7', 'G7', 'G-7', 'C7'] F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7 F^7
['F^7', 'Eh7', 'A7', 'D-7'] G7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7 C^7
['D-6', 'D-6', 'D-6', 'D-6'] D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7 D-7
